In [27]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.metrics import mean_squared_error

import sys
sys.path.append("..")

### Load data

In [12]:
train = pd.read_csv("../data/train.txt", sep="|", index_col=0)
val   = pd.read_csv("../data/val.txt", sep="|", index_col=0)

In [13]:
print(train.shape)
train.head()

(40335, 14)


,pm2_5,hour_sin,hour_cos,NE,NW,SE,SW,Fri,Mon,Sat,Sun,Thu,Tue,Wed
date,,,,,,,,,,,,,,
2010-01-02 00:00:00,129.0,0.000000,1.000000,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-02 01:00:00,148.0,0.269797,0.962917,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-02 02:00:00,159.0,0.519584,0.854419,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-02 03:00:00,181.0,0.730836,0.682553,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2010-01-02 04:00:00,138.0,0.887885,0.460065,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Build and train model

In [14]:
seed=421
target="pm2_5"

In [22]:
forecaster = ForecasterAutoreg(
                regressor = RandomForestRegressor(random_state=seed),
                lags      = 8
             )
forecaster.fit(y=train[target], exog=train.drop(columns = [target]).values)
forecaster

=======================ForecasterAutoreg=======================
Regressor: RandomForestRegressor(random_state=421)
Lags: [1 2 3 4 5 6 7 8]
Exogenous variable: True
Parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 421, 'verbose': 0, 'warm_start': False}

In [26]:
y_pred = forecaster.predict(steps=val.shape[0], exog=val.drop(columns = [target]).values)

In [28]:
error_rmse = mean_squared_error(
                y_true = val[target],
                y_pred = y_pred, squared=False
            )
print(f"Error validation (rmse): {error_rmse}")

Error validation (rmse): 98.66491670992669
